<h2>Table of content</h2>

<div style="margin-top: 20px">
<ol>
    <li><a href="#part1">Part 1. Gathering the data from wiki page and pre-processing</a></li>
    <li><a href="#part2">Part 2. Latitude and longitude</a></li>
</ol>
</div>
 
<hr>

<h2 id="part1">Part 1. Gathering the data from wiki page and pre-processing</h2>

In [2]:
#import libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
#let's download the html code of wiki webpage

wiki_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(wiki_page.content, 'html.parser')

In [4]:
#all the table data are in 'tr' elements, so get them all first

table_rows = soup.find_all('tr')

#create an empty list to fill it with the data

result = []

#exctract all the 'td' elements from the 'tr'. In addition we take data which starts from row with first letter "M" (to be sure that this is postal code). And one more filter - skipping 'Not Assigned' Boroughs

for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        if row[0][0] == 'M' and row[1] != 'Not assigned':
            result.append(row)

#now we have a list with the info we need. If a cell has a borough but a Not assigned neighborhood, we put the Neighbourhood name the same as the borough name

for row in result:
    if row[2] == 'Not assigned':
        row[2] = row[1]
        
#creating the dataframe with the data we got so far

toronto_df = pd.DataFrame(result, columns=["Postcode", "Borough", "Neighbourhood"])

In [5]:
#check the shape of DF

print(toronto_df.shape)
print(toronto_df.head())

(211, 3)
  Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M5A  Downtown Toronto       Regent Park
4      M6A        North York  Lawrence Heights


In [6]:
#create a new DF to populate it with unique Postcodes from the list "postcodes"

new_df = pd.DataFrame(columns=["Postcode", "Neighbourhood"])
postcodes = toronto_df['Postcode'].unique()

print(postcodes)

['M3A' 'M4A' 'M5A' 'M6A' 'M7A' 'M9A' 'M1B' 'M3B' 'M4B' 'M5B' 'M6B' 'M9B'
 'M1C' 'M3C' 'M4C' 'M5C' 'M6C' 'M9C' 'M1E' 'M4E' 'M5E' 'M6E' 'M1G' 'M4G'
 'M5G' 'M6G' 'M1H' 'M2H' 'M3H' 'M4H' 'M5H' 'M6H' 'M1J' 'M2J' 'M3J' 'M4J'
 'M5J' 'M6J' 'M1K' 'M2K' 'M3K' 'M4K' 'M5K' 'M6K' 'M1L' 'M2L' 'M3L' 'M4L'
 'M5L' 'M6L' 'M9L' 'M1M' 'M2M' 'M3M' 'M4M' 'M5M' 'M6M' 'M9M' 'M1N' 'M2N'
 'M3N' 'M4N' 'M5N' 'M6N' 'M9N' 'M1P' 'M2P' 'M4P' 'M5P' 'M6P' 'M9P' 'M1R'
 'M2R' 'M4R' 'M5R' 'M6R' 'M7R' 'M9R' 'M1S' 'M4S' 'M5S' 'M6S' 'M1T' 'M4T'
 'M5T' 'M1V' 'M4V' 'M5V' 'M8V' 'M9V' 'M1W' 'M4W' 'M5W' 'M8W' 'M9W' 'M1X'
 'M4X' 'M5X' 'M8X' 'M4Y' 'M7Y' 'M8Y' 'M8Z']


In [7]:
#add the postcode and all the Neighbourhoods belong to it into the "new_df" DataFrame

for code in postcodes:
    neighbourhoods = ", ".join(toronto_df[toronto_df['Postcode'] == code]['Neighbourhood'])
    new_df = new_df.append({"Postcode" : code, "Neighbourhood": str(neighbourhoods)}, ignore_index=True)
    
print(new_df.head())
print(new_df.shape)

  Postcode                     Neighbourhood
0      M3A                         Parkwoods
1      M4A                  Victoria Village
2      M5A         Harbourfront, Regent Park
3      M6A  Lawrence Heights, Lawrence Manor
4      M7A                      Queen's Park
(103, 2)


In [8]:
#let's drop the duplicated postcodes from the original DataFrame

toronto_df = toronto_df.drop_duplicates(subset=['Postcode'])

In [9]:
# finally we merge two df with unique Postcodes and all the related neighbourhoods combined
toronto_df = pd.merge(toronto_df, new_df, on = 'Postcode')
toronto_df = toronto_df.drop(['Neighbourhood_x'], axis = 1)
toronto_df = toronto_df.rename(columns={'Neighbourhood_y': ' Neighbourhood'})

In [10]:
print(toronto_df.head())

  Postcode           Borough                     Neighbourhood
0      M3A        North York                         Parkwoods
1      M4A        North York                  Victoria Village
2      M5A  Downtown Toronto         Harbourfront, Regent Park
3      M6A        North York  Lawrence Heights, Lawrence Manor
4      M7A      Queen's Park                      Queen's Park


In [11]:
print(toronto_df.shape)

(103, 3)


<h2 id="part2">Part 2. Latitude and longitude</h2>

In [21]:
# Create a new df to process with coordinates
df = toronto_df

In [13]:
import geocoder # this returns a mistake ModuleNotFoundError: No module named 'geocoder' so I decided to proceed with CSV file instead

ModuleNotFoundError: No module named 'geocoder'

In [22]:
#CSV reading
lat_long = pd.read_csv('Geospatial_Coordinates.csv')
print(lat_long.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476


In [23]:
#Rename the column "Postal code" to merge two dataframes
lat_long = lat_long.rename(columns={'Postal Code':'Postcode'})
#Merging
df = pd.merge(df, lat_long, on='Postcode')
df.head(12)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
